<a href="https://colab.research.google.com/github/m-adeleke1/Association_of_Data_Scientists/blob/main/RAG_Pinecone_URL_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

In [1]:
!pip install langchain-openai langchain-community unstructured==0.7.12 pinecone openai tiktoken -q
!pip install langchain langchain-pinecone -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 

## Importing Dependencies

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores.pinecone import Pinecone
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
import os
import nltk
from pinecone import Pinecone, ServerlessSpec
import openai
from google.colab import userdata

## User Input for URLs to build copilot on

In [4]:
urls = []
n_weblinks = int(input("How many web links you want the copilot to refer for response generation & insights? Enter here: "))
print("Enter your links below: ")
for i in range(0, n_weblinks):
  inp = input()
  # appending the element in list
  urls.append(inp)

How many web links you want the copilot to refer for response generation & insights? Enter here: 1
Enter your links below: 
https://arxiv.org/pdf/2312.10997


## Loading URL through LangChain's UnstructuredURLLoader

In [5]:
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(urls=urls)
urls = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
ERROR:langchain_community.document_loaders.url:Error fetching or processing https://arxiv.org/pdf/2312.10997, exception: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
*******************************************

In [6]:
# Merging urls into a single list
documents = []
documents.extend(urls)

In [7]:
documents

[]

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
texts

[]

## OpenAI API Key Setting



In [10]:
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Pinecone API Setting

In [11]:
#Set the pinecone key
pc = Pinecone(api_key = userdata.get("PINECONE_API_KEY"))
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

In [13]:
#Creating a Pinecone index automatically if index doesn't exists
from langchain_pinecone import PineconeVectorStore
embeddings = OpenAIEmbeddings()
index_name = 'myindex' #index name
if index_name not in pc.list_indexes():
    # Creating a new index
    pc.create_index(name=index_name, metric="cosine", dimension=1536,
                    vector_type="dense",
                    spec=ServerlessSpec(
                        cloud="aws",
                        region="us-east-1"
                    ))
# The OpenAI embedding model 'text-embedding-ada-002 uses 1536 dimensions'
docsearch = PineconeVectorStore.from_documents(texts, embeddings, index_name=index_name)

## Model Building and Execution

In [14]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4", max_tokens=256)
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=docsearch.as_retriever(),
        verbose=True,
        return_source_documents=True)

In [15]:
initial_prompt = "You are a researcher who is going to search the web links, summarize them and share insights as asked"

In [18]:
query = "What is a RAG framework?"
result = qa({"query": query, "prompt": initial_prompt})



> Entering new RetrievalQA chain...

> Finished chain.


In [19]:
print(result['result'])

A RAG framework, or Red Amber Green framework, is a system used in project management to visually indicate the status or performance of various tasks or goals. Each color represents a different status: red typically indicates a problem or issue, amber suggests there may be a potential issue or delay, and green signifies that everything is on track. This system allows for quick, at-a-glance assessments of a project's progress.


In [22]:
print(result['source_documents'])

[]
